In [1]:
pwd

'C:\\Users\\ASUS\\PycharmProjects\\ObjectDetection\\workspace\\training_demo'

In [1]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import tensorflow as tf

PATH_TO_SAVED_MODEL = "exported-models/my_model/saved_model/"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# %%
# Load label map data (for plotting)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Label maps correspond index numbers to category names, so that when our convolution network
# predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility
# functions, but anything that returns a dictionary mapping integers to appropriate string labels
# would be fine.

category_index = label_map_util.create_category_index_from_labelmap("annotations/label_map.pbtxt",
                                                                    use_display_name=True)

Loading model...Done! Took 15.639317750930786 seconds


In [2]:


# %%
# Putting everything together
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~
# The code shown below loads an image, runs it through the detection model and visualizes the
# detection results, including the keypoints.
#
# Note that this will take a long time (several minutes) the first time you run this code due to
# tf.function's trace-compilation --- on subsequent runs (e.g. on new images), things will be
# faster.
#
# Here are some simple things to try out if you are curious:
#
# * Modify some of the input images and see if detection still works. Some simple things to try out here (just uncomment the relevant portions of code) include flipping the image horizontally, or converting to grayscale (note that we still expect the input image to have 3 channels).
# * Print out `detections['detection_boxes']` and try to match the box locations to the boxes in the image.  Notice that coordinates are given in normalized form (i.e., in the interval [0, 1]).
# * Set ``min_score_thresh`` to other values (between 0 and 1) to allow more detections in or to filter out more detections.
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

IMAGE_PATHS = ["images/test/005697_jpg.rf.2dde27a7e8e8e6b6c4151ce287406865.jpg"]
for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
plt.show()



Running inference for images/test/005697_jpg.rf.2dde27a7e8e8e6b6c4151ce287406865.jpg... Done


In [3]:
detections['detection_scores'] > 0.30

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [4]:
detections["detection_boxes"]

array([[0.23061638, 0.19146803, 0.3495945 , 0.2475982 ],
       [0.2743924 , 0.07968281, 0.40632734, 0.15943505],
       [0.30971873, 0.84320956, 0.41479433, 0.90063006],
       [0.25444388, 0.4382939 , 0.37120777, 0.5017881 ],
       [0.27961102, 0.58511657, 0.3848959 , 0.6402753 ],
       [0.3203848 , 0.3712949 , 0.64430946, 0.5473875 ],
       [0.42733425, 0.858601  , 0.5206759 , 0.9033233 ],
       [0.19047338, 0.43198153, 0.36969876, 0.5253101 ],
       [0.256676  , 0.43845764, 0.3695623 , 0.50123304],
       [0.21400481, 0.57669085, 0.38628703, 0.65496284],
       [0.25428638, 0.5824373 , 0.38259473, 0.6446905 ],
       [0.2548258 , 0.6261557 , 0.8170928 , 0.9080908 ],
       [0.30882868, 0.84100425, 0.41636375, 0.899645  ],
       [0.23296782, 0.19030784, 0.35032353, 0.250798  ],
       [0.2312946 , 0.57883376, 0.38912836, 0.6504634 ],
       [0.4395481 , 0.8626361 , 0.5242481 , 0.9014684 ],
       [0.23159063, 0.17659472, 0.3837303 , 0.25863996],
       [0.16263995, 0.41382787,

In [5]:
coordinates = detections["detection_boxes"][detections['detection_scores'] > 0.30]

In [6]:
dw, dh, _ = image_np_with_detections.shape

In [7]:
for i in coordinates:
    i[0] = int(i[0] * dh)
    i[1] = int(i[1] * dw)
    i[2] = int(i[2] * dh)
    i[3] = int(i[3] * dw)
coordinates = coordinates.astype(np.int32)

In [8]:
coordinates

array([[115,  63, 174,  82],
       [137,  26, 203,  53],
       [154, 280, 207, 299],
       [127, 145, 185, 167],
       [139, 194, 192, 213],
       [160, 123, 322, 182],
       [213, 285, 260, 300],
       [ 95, 143, 184, 174],
       [128, 146, 184, 166],
       [107, 192, 193, 218]])

In [9]:
dw

333

In [10]:
image_np_new = image_np[91:135, 39:79]

In [12]:
import cv2
def display_img(img, rgb=False):
    while True:
        if rgb:
            new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        else:
            new_img = img
        cv2.imshow("image", new_img)
        if cv2.waitKey(0) or 0xFF == 27:
            cv2.destroyAllWindows()
            return -1
    return 0
    
display_img(image_np_new, rgb=True)

-1

In [13]:
cap = cv2.VideoCapture(0)
    
while True:
    ret, frame = cap.read()
    cv2.imshow("frame",frame)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



        

In [ ]:
vid = cv2.VideoCapture(0)
  
while(True):
    ret, frame = vid.read()
    image_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
    new_img = cv2.cvtColor(image_np_with_detections, cv2.COLOR_RGB2BGR)
    cv2.imshow('frame', new_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
vid.release()
cv2.destroyAllWindows()